Muhammad Aizar Yazid / 1103223097

### **Chapter 3: Classification**

#### **1. Ringkasan Teori dan Reproduksi Kode**

Chapter 3 berfokus pada tugas *supervised learning* yang paling umum kedua, yaitu **klasifikasi**. Tidak seperti regresi yang memprediksi nilai numerik, klasifikasi memprediksi kelas atau kategori dari sebuah instance. Bab ini menggunakan dataset **MNIST**, yang merupakan "hello world" dari Machine Learning, untuk mendemonstrasikan berbagai konsep klasifikasi.

##### **1.1 Dataset MNIST**

Dataset MNIST adalah kumpulan 70.000 gambar kecil tulisan tangan angka oleh siswa sekolah menengah dan karyawan Biro Sensus AS. Setiap gambar diberi label dengan angka yang diwakilinya. Scikit-Learn menyediakan fungsi pembantu untuk mengunduh dataset populer, termasuk MNIST.

```python
# Mengunduh dataset MNIST
from sklearn.datasets import fetch_openml
import numpy as np

mnist = fetch_openml('mnist_784', version=1)
X, y = mnist["data"], mnist["target"]
y = y.astype(np.uint8) # Mengubah label menjadi integer

# Memisahkan dataset menjadi training set dan test set
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
```

##### **1.2 Pelatihan Klasifikasi Biner**

Untuk menyederhanakan masalah, bab ini dimulai dengan membuat sebuah **binary classifier**, yaitu "5-detector" yang mampu membedakan antara dua kelas: angka 5 dan bukan angka 5.

```python
# Membuat target untuk klasifikasi biner
y_train_5 = (y_train == 5) # True untuk semua angka 5, False untuk lainnya
y_test_5 = (y_test == 5)

# Melatih model Stochastic Gradient Descent (SGD)
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)
```

##### **1.3 Ukuran Kinerja (Performance Measures)**

Mengevaluasi sebuah *classifier* seringkali lebih rumit daripada mengevaluasi *regressor*.

* **Akurasi Menggunakan Cross-Validation**
    Akurasi sering digunakan, tetapi bisa sangat menyesatkan, terutama pada *skewed datasets* (ketika beberapa kelas jauh lebih sering muncul daripada yang lain). Sebagai contoh, sebuah *classifier* bodoh yang selalu menebak "bukan-5" akan memiliki akurasi di atas 90% karena hanya sekitar 10% dari gambar adalah angka 5.

* **Matriks Konfusi (Confusion Matrix)**
    Cara yang jauh lebih baik untuk mengevaluasi kinerja adalah dengan melihat matriks konfusi. Matriks ini menunjukkan jumlah instance kelas A yang diklasifikasikan sebagai kelas B. Baris mewakili kelas aktual, sedangkan kolom mewakili kelas yang diprediksi.

    ```python
    from sklearn.model_selection import cross_val_predict
    from sklearn.metrics import confusion_matrix
    
    y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)
    cm = confusion_matrix(y_train_5, y_train_pred)
    # cm akan berisi [true_negatives, false_positives]
    #               [false_negatives, true_positives]
    print(cm)
    ```

* **Precision dan Recall**
    Matriks konfusi memberikan banyak informasi, tetapi terkadang metrik yang lebih ringkas lebih disukai.
    * **Precision**: Akurasi dari prediksi positif; `TP / (TP + FP)`.
    * **Recall** (atau *sensitivity*): Rasio instance positif yang terdeteksi dengan benar oleh *classifier*; `TP / (TP + FN)`.

    ```python
    from sklearn.metrics import precision_score, recall_score
    
    print("Precision:", precision_score(y_train_5, y_train_pred))
    print("Recall:", recall_score(y_train_5, y_train_pred))
    ```

* **F1 Score**
    F1 score adalah **rata-rata harmonik** dari *precision* dan *recall*. Rata-rata harmonik memberikan bobot lebih pada nilai yang rendah, sehingga *classifier* hanya akan mendapatkan skor F1 tinggi jika *recall* dan *precision*-nya sama-sama tinggi.

    ```python
    from sklearn.metrics import f1_score
    
    print("F1 Score:", f1_score(y_train_5, y_train_pred))
    ```

* **Precision/Recall Trade-off**
    Meningkatkan *precision* cenderung mengurangi *recall*, dan sebaliknya. Ini disebut *precision/recall trade-off*. Kita bisa memilih *threshold* yang berbeda pada *decision function* model untuk mendapatkan trade-off yang diinginkan.

* **Kurva ROC (Receiver Operating Characteristic)**
    Kurva ROC memplot **true positive rate (recall)** terhadap **false positive rate (FPR)**. FPR adalah rasio instance negatif yang salah diklasifikasikan sebagai positif. Kurva ini adalah alat umum lain yang digunakan untuk *binary classifier*. Semakin dekat kurva ke sudut kiri atas, semakin baik *classifier*-nya. **Area Under the Curve (AUC)** adalah ukuran kinerja yang umum: AUC 1 berarti *classifier* sempurna, sementara 0.5 berarti *classifier* acak.

##### **1.4 Klasifikasi Multikelas**

*Classifier* multikelas (juga disebut multinomial) dapat membedakan antara lebih dari dua kelas. Beberapa algoritma seperti `RandomForestClassifier` atau `SGDClassifier` dapat menangani beberapa kelas secara native. Algoritma lain seperti `SVC` adalah *binary classifier* murni. Namun, Scikit-Learn mendeteksi ketika Anda mencoba menggunakan algoritma klasifikasi biner untuk tugas klasifikasi multikelas, dan secara otomatis menjalankan strategi **One-vs-the-Rest (OvR)** atau **One-vs-One (OvO)**.

##### **1.5 Analisis Kesalahan**

Setelah menemukan model yang menjanjikan, cara untuk meningkatkannya adalah dengan menganalisis jenis kesalahan yang dibuatnya.
1.  **Analisis Matriks Konfusi**: Dengan memplot matriks konfusi sebagai gambar, kita dapat dengan mudah melihat kelas mana yang sering salah diklasifikasikan. Normalisasi matriks konfusi berdasarkan jumlah gambar di setiap kelas akan memberikan tingkat kesalahan, bukan jumlah kesalahan absolut.
2.  **Analisis Kesalahan Individual**: Menganalisis contoh-contoh spesifik yang salah diklasifikasikan dapat memberikan wawasan berharga tentang kelemahan model (misalnya, sensitif terhadap pergeseran atau rotasi gambar).

##### **1.6 Klasifikasi Multilabel dan Multioutput**

* **Multilabel Classification**: Sebuah sistem klasifikasi yang dapat mengeluarkan beberapa *tag* biner untuk setiap instance. Contohnya, sebuah *classifier* mengenali beberapa orang dalam satu gambar.
* **Multioutput Classification**: Generalisasi dari *multilabel classification* di mana setiap label bisa berupa multikelas (yaitu, dapat memiliki lebih dari dua nilai yang mungkin). Contohnya adalah sistem penghilang *noise* pada gambar, di mana outputnya adalah semua nilai intensitas piksel.

---
#### **2. Latihan (Exercises)**

**1. Bangun sebuah *classifier* untuk dataset MNIST yang mencapai akurasi di atas 97% pada *test set*.**

`KNeighborsClassifier` bekerja cukup baik untuk tugas ini. Kita bisa menggunakan `GridSearchCV` untuk menemukan *hyperparameter* terbaik (`weights` dan `n_neighbors`).

```python
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# Kode ini akan berjalan sangat lama
# param_grid = [{'weights': ["uniform", "distance"], 'n_neighbors': [3, 4, 5]}]
# knn_clf = KNeighborsClassifier()
# grid_search = GridSearchCV(knn_clf, param_grid, cv=5, verbose=3)
# grid_search.fit(X_train, y_train)

# best_params = grid_search.best_params_ # -> {'n_neighbors': 4, 'weights': 'distance'}
# accuracy = grid_search.best_score_ # -> Mencapai ~97%
```
Dengan `n_neighbors=4` dan `weights='distance'`, akurasi di atas 97% dapat dicapai.

**2. Tulis sebuah fungsi yang dapat menggeser gambar MNIST ke segala arah (kiri, kanan, atas, bawah) sejauh satu piksel.**

Kita dapat menggunakan fungsi `shift` dari `scipy.ndimage.interpolation`. Kemudian, untuk setiap gambar di *training set*, kita buat empat salinan yang digeser dan tambahkan ke *training set*. Ini disebut **data augmentation**.

```python
from scipy.ndimage.interpolation import shift

def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])

# Contoh penggunaan
# X_train_augmented = [image for image in X_train]
# y_train_augmented = [label for label in y_train]

# for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
#     for image, label in zip(X_train, y_train):
#         X_train_augmented.append(shift_image(image, dx, dy))
#         y_train_augmented.append(label)

# X_train_augmented = np.array(X_train_augmented)
# y_train_augmented = np.array(y_train_augmented)
```
Melatih model pada dataset yang diperluas ini akan meningkatkan kinerjanya karena model menjadi lebih toleran terhadap pergeseran posisi.

**3. Kerjakan dataset Titanic.**

Dataset Titanic adalah dataset klasifikasi klasik. Langkah-langkahnya adalah:
1.  Unduh data dari Kaggle atau menggunakan `fetch_openml`.
2.  Lakukan eksplorasi data untuk memahami fitur-fitur seperti `Pclass`, `Sex`, `Age`, `SibSp`, `Parch`, dan `Fare`.
3.  Lakukan pra-pemrosesan data: isi nilai `Age` yang hilang (misalnya dengan median), ubah fitur kategorikal (`Sex`) menjadi numerik, dan isi nilai `Embarked` yang hilang.
4.  Pilih model klasifikasi seperti `RandomForestClassifier` atau `SVC`.
5.  Latih model dan evaluasi menggunakan *cross-validation* untuk melihat kinerjanya.

**4. Buat sebuah *spam classifier*.**

Ini adalah latihan yang lebih menantang yang melibatkan NLP. Langkah-langkah utamanya adalah:
1.  Unduh contoh spam dan ham (misalnya dari dataset publik Apache SpamAssassin).
2.  Bagi dataset menjadi *training set* dan *test set*.
3.  Tulis *pipeline* pra-pemrosesan data untuk mengubah setiap email menjadi vektor fitur. Ini melibatkan:
    * Mengubah email menjadi *lowercase*.
    * Menghapus tanda baca.
    * Mengganti URL dan angka dengan token khusus ("URL", "NUMBER").
    * Melakukan *stemming* (memotong akhir kata).
    * Mengubah email menjadi vektor *bag-of-words* (misalnya dengan `CountVectorizer` atau `TfidfVectorizer`).
4.  Latih beberapa *classifier* seperti `LogisticRegression`, `SVC`, atau `RandomForestClassifier` dan bandingkan kinerjanya untuk mendapatkan *precision* dan *recall* yang tinggi.